This gives the rationale and some of the descriptive statistics for the preprocessing done in <preprocessing.ipynb>.
It should do the same preprocessing, but will also give the descriptive stats along the way.

In [ ]:
# functions and packages

import math
import numpy as np
import re
import string
import scipy
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import datetime as dt

#this recodes WeapRecode1 and WeapRecode2 to specs in weaptype recode.xlsx
def weap_recode(wtype,wsubtype):
    if (wtype in (1,2,3,4)):
        ret = wtype
    if (wtype == 5):
        ret = wsubtype + 3
    if (wtype == 6):
        if (wsubtype < 28):
            ret = wsubtype + 3
        else:
            ret = wsubtype - 7
    if (wtype == 7):
        ret = 23
    if (wtype == 8):
        ret = wsubtype + 6
    if (wtype == 9):
        if (wsubtype < 26):
            ret = wsubtype + 6
        else:
            ret = wsubtype + 5
    if (wtype in (10,11,12,13)):
        ret = wtype + 23
    if (pd.isnull(wtype)):
        ret = np.NaN
    return ret

def c_weekday(y,m,d):
    if (m != 0 and d !=0):
        t_dt = dt.datetime(y,m,d)
        ret = t_dt.weekday()
    else:
        ret = np.nan
    return ret
    
# this recodes propextent to:
# 0 = no damage
# 1 = Minor (likely < $1 million)
# 2 = Major (likely > $1 million but < $1 billion)
# 3 =  Catastrophic (likely > $1 billion)       
# Nan = unknown

def prop_extent_recode(prop,prop_ext):
    if (prop_ext == 4):
        ret = np.NaN
    if (prop == 0):
        ret = 0
    if (prop_ext in (1,2,3)):
        if (pd.isnull(prop)):
            ret = np.NaN
        else:
            ret = 4 - prop_ext
    if (pd.isnull(prop_ext)):
        ret = np.NaN
    return ret


In [ ]:
# 1. Add Weekday

# It seemed that weekday could predict terrorist attacks. For example, 
# the difference in Saturday and Sunday for Judiasm and Christianity

home = r"C:\Users\ibshi\Desktop\startup.ml\challenge 2\global terrorism\data"
infile = home + r"\globalterrorismdb_0616dist.xlsx"
indata = pd.read_excel(infile)


    # look at days and months distributions, to see if calculating 
    # weekdays would be an issue. Some missing values, so a check for zeroes
    # in c_weekday

np.histogram(indata.iday,bins=31) # 894 zeroes
np.histogram(indata.imonth,bins=12) # 23 zeroes

    # plot histogram of Days (days.jpg)
    
plt.hist(indata.iday,bins=31)
plt.title("Days")
plt.xlabel("Days")
plt.ylabel("Frequency")

figfile = home + r"\days.jpg"
plt.savefig(figfile)

    # plot histogram of Months (months.jpg)
    
plt.hist(indata.imonth,bins=12)
plt.title("Months")
plt.xlabel("Months")
plt.ylabel("Frequency")

figfile = home + r"\months.jpg"
plt.savefig(figfile)

# run code to transform

weekday_data = indata
weekday_data.insert(5,'Weekday',-1)

for i in list(weekday.index):
    if (i % 1000 == 0):
        print(i)
    t_year = weekday_data.iyear[i]
    t_month = weekday_data.imonth[i]
    t_day = weekday_data.iday[i]
    weekday_data.Weekday[i] = c_weekday(t_year,t_month,t_day)



In [ ]:
# 2. remove rows where terrorism status is in doubt
#    The field 'doubtterr' asks: Is there doubt that
#    this is this an act of terrorism? (1=yes, 0=no).

# This gives 24227 cases of terrorism status is in doubt
np.histogram(indata.doubtterr,range = (-9,2),bins = 11)

# with 156772 entries beforehand, removing doubtterr == 1
# leaves 132545 entries

weekday_data = weekday_data[weekday_data.doubtterr!=1]
weekday_data.info()

# save to intermediary file
    
weekdayfile = home + r"\weekday.xlsx"
weekday_data.to_excel(weekdayfile)
 

In [ ]:
# 3. remove guncertain == 1, keeping ==0 and ==NaN
#    Is the terrorism group identity uncertain?
#    (1=yes, 0=no)

# read intermediary file
infile = home + r"\weekday.xlsx"
weekday = pd.read_excel(infile)

# From 132545 entries,
# this leaves 119965 entries left in weekday
weekday = weekday[weekday.guncertain1 != 1]
weekday.info()


In [ ]:
# 4. if competing claim, set 'claimed' to NaN
#    the field 'claimed' indicates whether the the terrorist attack was
#    claimed by a terorist group. (1=yes, 0=no)
#    If compclaim ==1, then there was a competing claim by another
#    terrorist group. If compclaim ==1, claimed was set to NaN

# This gives compclaim ==1, 138 entries, competing claims for attack

t = weekday[weekday.compclaim == 1]
t.info()

weekday.claimed[weekday.compclaim == 1] = np.NaN



In [ ]:
# 5. recode WeapRecode1, WeapRecode2
#    The fields weaptype1 and weaptype2 give broader categories for the
#    weapon type for the attack.  The fields weapsubtype1 and weapsubtype2
#    give a more detailed description of the weapon types.  However, some
#    of the weapontypes have no subtypes (e.g., nuclear, chemical).  Also,
#    the weapon types of firearms and explosives/gun/dynamite have several
#    subtypes and counts of approximately 36K and 63K (for weaptype1)
#    Thus, two new fields were created, WeapRecode1 and WeapRecode2, that
#    combined weaptype and weapsubtype into one field. The codes can be 
#    found in weaptype recode.xlsx

# Codes for weaptype

# 1 = Biological
# 2 = Chemical
# 3 = Radiological
# 4 = Nuclear
# 5 = Firearms
# 6 = Explosives/Bombs/Dynamite
# 7 = Fake Weapons
# 8 = Incendiary
# 9 = Melee
# 10 = Vehicle
# 11 = Sabotage Equipment
# 12 = Other
# 13 = Unknown

# codes for weapsubtype for those categories having multiple subtypes
# all other weaptypes do not have 

# Firearms
# 2 = Automatic Weapon (including semi‐automatic)
# 3 = Handgun
# 4 = Rifle/Shotgun (non‐automatic)
# 5 = Unknown Gun Type
# 6 = Other Gun Type
# Explosives/Bombs/Dynamite
# 7 = Grenade (not RPGs)
# 8 = Mine
# 9 = Mail Bomb (letter, package, parcel)
# 10 = Pressure Trigger
# 11 = Projectile (e.g., rockets, mortars, RPGs, missiles)
# 12 = Remote Device (trigger, control, detonate)
# 13 = Suicide (carried bodily by human being)
# 14 = Time Fuse
# 15 = Vehicle
# 16 = Unknown Explosive Type
# 17 = Other Explosive Type
# 28 = Dynamite/TNT
# 29 = Sticky Bomb
# Incendiary
# 18 = Arson/Fire
# 19 = Molotov Cocktail/Petrol Bomb
# 20 = Gasoline or Alcohol
# Melee
# 21 = Blunt Object
# 22 = Hands, Feet, Fists
# 23 = Knife or Other Sharp Object
# 24 = Rope or Other Strangling Device
# 26 = Suffocation
# 27 = Unknown Weapon Type

#
# the following indicate 
# weaptype1 == 'Firearms' has 35933 entries
# weaptype1 == 'Explosives/Bombs/Dynamite' has 65645 entries
# weaptype2 == 'Firearms' has 2686 entries
# weaptype2 == 'Explosives/Bombs/Dynamite' has 1876 entries
#

l=np.histogram(weekday.weaptype1,range=(1,14),bins = 13)
#[21,183,12,0,35933,63645,28,8270,2366, 67,86,79,9275]
l=np.histogram(weekday.weaptype2,range=(1,14),bins = 13)
#[1,69,0,0,2686,1876,10,1541,818,31,38,75,288]

weekday.insert(93,'WeapRecode2',-1)
weekday.insert(88,'WeapRecode1',-1)

for i in list(weekday.index):
    w = weekday.weaptype1[i]
    ws = weekday.weapsubtype1[i]
    weekday.WeapRecode1[i] = weap_recode(w,ws)
    w = weekday.weaptype2[i]
    ws = weekday.weapsubtype2[i]
    weekday.WeapRecode2[i] = weap_recode(w,ws)


In [ ]:
# 6. recode propextent so that property ==0 is included.  Also values switched
#    so 0 to 3 increase in value
#    The field 'property' asks the question if there was property damage 
#    (1=yes,0=no). Also, the coding for propextent has 1 as the most 
#    extensive damage and 3 as the least extensive, and 4 = Unknown.
#    The field propextent was recoded and combined with property so that
#    0 = no damage
#    1 = = Minor (likely < $1 million)
#    2 = Major (likely > $1 million but < $1 billion)
#    3 = Catastrophic (likely > $1 billion)
#    NaN = Unknown

#    This code shows that there were 249 entries that had property = 0 
#    (no prop damange) and propextent > 0, which shouldn't happen

t = weekday[(weekday.property == 0) & (weekday.propextent > 0)]
l=np.histogram(t.propextent,range=(0,5),bins = 5)

#    there were 167 entries with propextent = 3 (minor), 
#    82 with propextent = 4 (unknown)

#    doing the conditionals in the order found in prop_extent_recode 
#    means that:
#    if property ==0 and propextent ==3, propextent == 3 (minor) takes precedent
#    if property ==0 and propextent ==4, property ==0 takes precedent

for i in list(weekday.index):
    p = weekday.property[i]
    pe = weekday.propextent[i]    
    weekday.propextent[i] = prop_extent_recode(p,pe)
    

In [ ]:
# 7. recode nhostkid. If ishostkid == 0, set nhostkid =0
#    The field ishostkid asks the question 'Were there hostages or kidnap
#    victims?' (1=yes,0=no). The field nohostkid gives the number of
#    hostage/kidnap victims.  nhostkid was recoded so that if 
#    ishostkid == 0, nhostkid =0.

l=np.histogram(weekday.ishostkid,range=(-9,2),bins = 11)
np.sum((147,0,0,0,0,0,0,0,0, 110741,8948))
# ishostkid == 0, 110741 entries
# ishostkid == 1, 8948 entries

weekday.nhostkid.count()
# 8944 

weekday.nhostkid[weekday.ishostkid == 0] = 0


In [ ]:
# 8. nhours and ndays
#    the field ndays is rounded to nearest day. the field nhours is supposed to
#    be between 0 and 24.  into ndays so that
#    it is also rounded to nearest day

t = weekday[weekday.nhours > 24.0]
t.info()
#    nhours is supposed to be less than 24 hours
#    The above code gave 23 entries, mostly 999, I'm assuming that's a NaN code
#    the preprocessing code first checks for nhours = 999, then converts nhours
#    into ndays.

weekday.ndays[weekday.nhours ==999.0] = np.NaN
weekday.ndays[pd.notnull(weekday.nhours)] = np.round(weekday.nhours/24.0)


In [ ]:
# 9. motive had many 'unknown' text codes, change to NaN
#    This is a text field giving the motive, if known. motive was not used
#    in the final analysis, so this transform had no effect.

# count before taking out 'unknown' text values
weekday.motive.describe()
#count                                              33385
#unique                                              8524

# code to take two 'unknown' values out
t = weekday
t.motive[t.motive=="The specific motive for the attack is unknown."] = np.NaN
t.motive[t.motive=="Unknown"] = np.NaN
weekday = t

# count after taking out the two 'unknown' codes
weekday.motive.describe()
#count                                                 12646
#unique                                                 8522

# save to intermediary file weekday10.xlsx

weekday10file = home + r"\weekday10.xlsx"
weekday.to_excel(weekday10file)


In [ ]:
# 10. select features for analysis (into dataframe final_data) 
#     The final set of features are in data_final_list.xlsx

#     For some features there were equivalent text and numerical
#     fields. The text fields were dropped.  

#     For some types of features, more than one of that type of
#     feature could be listed for one attack (for example, 
#     targtype1, targtype2, etc.)  The non-primary features
#     tended to be less populated and therefore generally were 
#     dropped.

#     Only country and region were included from the location
#     information. 

home = r"C:\Users\ibshi\Desktop\startup.ml\challenge 2\global terrorism\data"
infile = home + r"\weekday10.xlsx"
weekday = pd.read_excel(infile)
final_data = weekday

droplist =['approxdate','extended','resolution','country_txt','region_txt', \
'provstate','city','latitude','longitude','specificity','vicinity', \
'location','summary','doubtterr','alternative','alternative_txt', \
'attacktype1_txt','attacktype2_txt','attacktype3','attacktype3_txt',\
'targtype1_txt', 'targsubtype1_txt','corp1','target1','natlty1','natlty1_txt', \
'targtype2','targtype2_txt','targsubtype2','targsubtype2_txt','corp2', \
'target2','natlty2','natlty2_txt','targtype3','targtype3_txt','targsubtype3', \
'targsubtype3_txt','corp3','target3','natlty3','natlty3_txt','gsubname', \
'gname2','gsubname2','gname3','ingroup','ingroup2','ingroup3','gsubname3', \
'motive','guncertain1','guncertain2','guncertain3','claimmode_txt', \
'claim2','claimmode2','claimmode2_txt','claim3','claimmode3', \
'claimmode3_txt','compclaim','weaptype1_txt','weapsubtype1', \
'weapsubtype1_txt','weaptype2_txt','weapsubtype2','weapsubtype2_txt', \
'weaptype3','weaptype3_txt','weapsubtype3','weapsubtype3_txt','weaptype4', \
'weaptype4_txt','weapsubtype4','weapsubtype4_txt','weapdetail', 'nkillus', \
'nkillter','nwoundus','nwoundte','property','propextent_txt','propvalue', \
'propcomment','ishostkid','nhostkidus','nhours','divert','kidhijcountry', \
'ransomamt','ransomamtus','ransompaid','ransompaidus','ransomnote', \
'hostkidoutcome_txt','addnotes','scite1','scite2','scite3','dbsource', \
'INT_ANY','related']

final_data.drop(droplist,inplace=True,axis=1)


In [ ]:
# 11. remove rows with gname == 'Unknown'
#     gname was the outcome to predict; therefore 'unknown'
#     gnames were removed

final_data.info()
#119965 entries

# after eliminating 'unknown' gnames
final_data = final_data[final_data.gname !='Unknown']
final_data.info()
# 57343 entries


In [ ]:
# 12. Choose gname with entries (rows) >= 150
#     This left 63 gnames as the number of categories to predict
#     from the original 2865

#     The code below gives the number of unique gnames when filtering
#     for the number of entries (rows) for each gname. That is,
#     how many gnames are there that have at least a certain number 
#     entries?  The number of entries tested below (more than):
#     1,2,3,4,5,6,7,8,9,14,19,24,49,99,149,199,249

#     From these data gnames with entries (rows) >= 150 was chosen,4
#     giving 63 unique gnames as outcomes
#     and 38251 total entries



t = final_data.gname
temp = t.value_counts()
temp.describe()
#count    2865.000000
#mean       20.015009
#std       142.671819

temp1 = temp[temp > 1]
temp1.describe()
#count    1429.000000
#mean       39.123163
temp2 = temp[temp > 2]
temp2.describe()
#count    1028.000000
#mean       53.604086
temp3 = temp[temp > 3]
temp3.describe()
#count     825.000000
#mean       66.055758
temp4 = temp[temp > 4]
temp4.describe()
#count     697.000000
#mean       77.451937
temp5 = temp[temp > 5]
temp5.describe()
#count     620.000000
#mean       86.450000
temp6 = temp[temp > 6]
temp6.describe()
#count     557.000000
#mean       95.549372
temp7 = temp[temp > 7]
temp7.describe()
#count     492.000000
#mean      107.247967
temp8 = temp[temp > 8]
temp8.describe()
#count     463.000000
#mean      113.464363
temp9 = temp[temp > 9]
temp9.describe()
#count     427.000000
#mean      122.271663
temp14 = temp[temp > 14]
temp14.describe()
#count     337.000000
#mean      151.703264
temp19 = temp[temp > 19]
temp19.describe()
#count     278.000000
#mean      180.266187
temp24 = temp[temp > 24]
temp24.describe()
#count     240.000000
#mean      205.320833
temp49 = temp[temp > 49]
temp49.describe()
#count     153.000000
#mean      301.169935
temp99 = temp[temp > 99]
temp99.describe()
#count      94.000000
#mean      447.414894
temp149 = temp[temp > 149]
temp149.describe()
#count      63.000000
#mean      607.158730
temp199 = temp[temp > 199]
temp199.describe()
#count      46.000000
#mean      768.934783
temp249 = temp[temp > 249]
temp249.describe()
#count      37.000000
#mean      902.594595


    # This finds a list of terrorist group names with >149 entries
t = final_data.gname
temp = t.value_counts()
temp149 = temp[temp > 149]
terrlist = list(temp149.index)

    # This inserts an 'inlist' column, figures out if gname is inlist
final_data.insert(0,'inlist',-1)
for i in list(final_data.index):
    final_data.inlist[i] = final_data.gname[i] in terrlist

    # This removes entries !inlist and removes inlist column
final_data = final_data[final_data.inlist==1]
final_data.drop('inlist',inplace=True,axis=1)

# create final files for analyses, Y_in.csv, final_data.xlsx

Y_in = final_data['gname']
final_data.drop(['gname','eventid'],inplace=True,axis=1)
outfile = home + r"\final_data.xlsx"
final_data.to_excel(outfile)
outfile = home + r"\Y_in.csv"
np.savetxt(outfile,Y_in,delimiter=",", fmt='%s')
